Notebook for Week 3 Capstone project

In [5]:
import pandas as pd
import numpy as np


Week 3 Part 1: To create a dataframe of Toronto Neighborhoods

In [34]:
#import data from Wikipedia
neighborhood_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
#read the list

df_list = pd.read_html(neighborhood_url) 

#create datafrom from list

toronto_df = pd.DataFrame (df_list[0]) 

#Remove rows where a borough is Not Assigned

toronto_df.drop(toronto_df[toronto_df['Borough'] == 'Not assigned'].index, inplace = True)
toronto_df

#To reset dataframe index

toronto_df = toronto_df.reset_index(drop=True) 

toronto_df.head() 

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [33]:
number_of_rows=toronto_df.shape[0]
print ('Number of rows =', number_of_rows)

Number of rows = 103
